In [1]:
# import os
# import sys

# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"


In [2]:
from numba import njit
import numpy as np
from nb_quantfreedom.utils import pretty_qf
from my_stuff import MufexKeys
from nb_quantfreedom.nb_enums import CandleBodyType
from nb_quantfreedom.nb_order_handler.nb_stop_loss import nb_Long_SLBasedOnCandleBodyCalc, nb_StopLoss
from nb_quantfreedom.nb_order_handler.nb_price_getter import nb_GetMinPrice, nb_PriceGetter
from nb_quantfreedom.nb_order_handler.nb_increase_position import (
    nb_Long_RPAandSLB,
    nb_IncreasePosition,
)
from nb_quantfreedom.nb_order_handler.nb_leverage import nb_CalcDynamicLeverage, nb_Leverage
from nb_quantfreedom.nb_order_handler.nb_take_profit import nb_TakeProfit,nb_TPRiskReward
from nb_quantfreedom.nb_enums import OrderResult


from nb_quantfreedom.exchanges.mufex_exchange.mufex import Mufex
mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2


In [3]:
candles = mufex_main.get_candles_df(
    symbol="BTCUSDT",
    timeframe="1m",
)


It took 00 mins and 00 seconds to download 200 candles


In [4]:
asset_tick_step = 3
available_balance = 1000.0
average_entry = 0
bar_index = 30
based_on_lookback = 20
cash_borrowed = 0
cash_used = 0
equity = 1000
exit_price = 0
fees_paid = 0
indicator_settings_index = 0
leverage_tick_step = 2
market_fee_pct = 0.0009
max_asset_size = 100
max_equity_risk_pct = 0.03
max_leverage = 150
max_trades = 3
min_asset_size = 0.001
mmr_pct = 0.005
order_settings_index = 0
position_size_asset = 0
position_size_usd = 0
possible_loss = 0
price_tick_step = 1
realized_pnl = 0
risk_account_pct_size = 0.03
risk_reward = 3
sl_based_on_add_pct = 0.005
static_leverage = 0
timestamp = int(candles[30, CandleBodyType.Timestamp])
total_trades = 0
tp_fee_pct = 0.0003

In [5]:
sl_calc = nb_Long_SLBasedOnCandleBodyCalc()
price_getter = nb_GetMinPrice()
calc_increase_pos = nb_Long_RPAandSLB()
lev_calc = nb_CalcDynamicLeverage()
tp_calculator = nb_TPRiskReward()

In [6]:
@njit()
def sl_calculator(
    bar_index: int,
    based_on_lookback: int,
    candle_body: int,
    candles: np.array,
    price_getter: nb_PriceGetter,
    price_tick_step: float,
    sl_based_on_add_pct: float,
    sl_calc: nb_StopLoss,
):
    sl_price = sl_calc.nb_sl_calculator(
        bar_index=bar_index,
        based_on_lookback=based_on_lookback,
        candle_body=candle_body,
        candles=candles,
        price_getter=price_getter,
        price_tick_step=price_tick_step,
        sl_based_on_add_pct=sl_based_on_add_pct,
    )
    return sl_price

In [7]:
@njit()
def increase_pos(
    calc_increase_pos: nb_IncreasePosition,
    account_state_equity: float,
    asset_tick_step: float,
    average_entry: float,
    entry_price: float,
    in_position: float,
    market_fee_pct: float,
    max_asset_size: float,
    max_equity_risk_pct: float,
    max_trades: int,
    min_asset_size: float,
    position_size_asset: float,
    position_size_usd: float,
    possible_loss: float,
    price_tick_step: float,
    risk_account_pct_size: float,
    sl_price: float,
    total_trades: int,
):
    (
        average_entry,
        entry_price,
        entry_size_asset,
        entry_size_usd,
        position_size_asset,
        position_size_usd,
        possible_loss,
        total_trades,
        sl_pct,
    ) = calc_increase_pos.calc_increase_position(
        account_state_equity=account_state_equity,
        asset_tick_step=asset_tick_step,
        average_entry=average_entry,
        entry_price=entry_price,
        in_position=in_position,
        market_fee_pct=market_fee_pct,
        max_asset_size=max_asset_size,
        max_equity_risk_pct=max_equity_risk_pct,
        max_trades=max_trades,
        min_asset_size=min_asset_size,
        position_size_asset=position_size_asset,
        position_size_usd=position_size_usd,
        possible_loss=possible_loss,
        price_tick_step=price_tick_step,
        risk_account_pct_size=risk_account_pct_size,
        sl_price=sl_price,
        total_trades=total_trades,
    )
    return (
        average_entry,
        entry_price,
        entry_size_asset,
        entry_size_usd,
        position_size_asset,
        position_size_usd,
        possible_loss,
        total_trades,
        sl_pct,
    )

In [8]:
@njit
def calc_lev(
    available_balance: float,
    average_entry: float,
    cash_borrowed: float,
    cash_used: float,
    entry_size_usd: float,
    lev_calc: nb_Leverage,
    leverage_tick_step: float,
    max_leverage: float,
    mmr_pct: float,
    sl_price: float,
    static_leverage: float,
    price_tick_step: float,
):
    return lev_calc.leverage_calculator(
        available_balance=available_balance,
        average_entry=average_entry,
        cash_borrowed=cash_borrowed,
        cash_used=cash_used,
        entry_size_usd=entry_size_usd,
        leverage_tick_step=leverage_tick_step,
        max_leverage=max_leverage,
        mmr_pct=mmr_pct,
        sl_price=sl_price,
        static_leverage=static_leverage,
        price_tick_step=price_tick_step,
    )

In [9]:
@njit
def calc_tp(
    average_entry: float,
    market_fee_pct: float,
    position_size_usd: float,
    possible_loss: float,
    price_tick_step: float,
    risk_reward: float,
    tp_calculator: nb_TakeProfit,
    tp_fee_pct: float,
):
    return tp_calculator.calculate_take_profit(
        average_entry=average_entry,
        market_fee_pct=market_fee_pct,
        position_size_usd=position_size_usd,
        possible_loss=possible_loss,
        price_tick_step=price_tick_step,
        risk_reward=risk_reward,
        tp_fee_pct=tp_fee_pct,
    )

In [10]:
sl_price = sl_calculator(
    bar_index=bar_index,
    based_on_lookback=based_on_lookback,
    candle_body=CandleBodyType.Low,
    candles=candles,
    price_getter=price_getter,
    price_tick_step=price_tick_step,
    sl_based_on_add_pct=sl_based_on_add_pct,
    sl_calc=sl_calc,
)

3 price min = <object type:float64>
sl_price = <object type:float64>


In [11]:
(
    average_entry,
    entry_price,
    entry_size_asset,
    entry_size_usd,
    position_size_asset,
    position_size_usd,
    possible_loss,
    total_trades,
    sl_pct,
) = increase_pos(
    calc_increase_pos=calc_increase_pos,
    account_state_equity=equity,
    asset_tick_step=asset_tick_step,
    average_entry=average_entry,
    entry_price=candles[2, CandleBodyType.Close],
    in_position=False,
    market_fee_pct=market_fee_pct,
    max_asset_size=max_asset_size,
    max_equity_risk_pct=max_equity_risk_pct,
    max_trades=max_trades,
    min_asset_size=min_asset_size,
    position_size_asset=position_size_asset,
    position_size_usd=position_size_usd,
    possible_loss=possible_loss,
    price_tick_step=price_tick_step,
    risk_account_pct_size=risk_account_pct_size,
    sl_price=sl_price,
    total_trades=total_trades,
)

Calculating
Possible Loss is fine


In [12]:
(
    available_balance,
    can_move_sl_to_be,
    cash_borrowed,
    cash_used,
    leverage,
    liq_price,
) = calc_lev(
    available_balance=available_balance,
    average_entry=average_entry,
    cash_borrowed=cash_borrowed,
    cash_used=cash_used,
    entry_size_usd=entry_size_usd,
    lev_calc=lev_calc,
    leverage_tick_step=leverage_tick_step,
    max_leverage=max_leverage,
    mmr_pct=mmr_pct,
    sl_price=sl_price,
    static_leverage=static_leverage,
    price_tick_step=price_tick_step,
)

In [13]:
tp_price, tp_pct, order_status = calc_tp(
    average_entry=average_entry,
    market_fee_pct=market_fee_pct,
    position_size_usd=position_size_usd,
    possible_loss=possible_loss,
    price_tick_step=price_tick_step,
    risk_reward=risk_reward,
    tp_calculator=tp_calculator,
    tp_fee_pct=tp_fee_pct,
)

In [14]:
order_result = OrderResult(
    indicator_settings_index=indicator_settings_index,
    order_settings_index=order_settings_index,
    bar_index=bar_index,
    timestamp=timestamp,
    equity=equity,
    available_balance=available_balance,
    cash_borrowed=cash_borrowed,
    cash_used=cash_used,
    average_entry=average_entry,
    fees_paid=fees_paid,
    leverage=leverage,
    liq_price=liq_price,
    order_status=order_status,
    possible_loss=possible_loss,
    entry_size_asset=entry_size_asset,
    entry_size_usd=entry_size_usd,
    entry_price=entry_price,
    exit_price=exit_price,
    position_size_asset=position_size_asset,
    position_size_usd=position_size_usd,
    realized_pnl=realized_pnl,
    sl_pct=sl_pct,
    sl_price=sl_price,
    total_trades=total_trades,
    tp_pct=tp_pct,
    tp_price=tp_price,
)
pretty_qf(order_result)

OrderResult(
    indicator_settings_index = 0,
    order_settings_index = 0,
    bar_index = 30,
    timestamp = 1697654340000,
    equity = 1000,
    available_balance = 929.9408,
    cash_borrowed = 3919.8142,
    cash_used = 70.0592,
    average_entry = 28335.9,
    fees_paid = 0,
    leverage = 85.34,
    liq_price = 28145.5,
    order_status = 1,
    possible_loss = 30,
    entry_size_asset = 0.141,
    entry_size_usd = 3989.8734,
    entry_price = 28335.9,
    exit_price = 0,
    position_size_asset = 0.141,
    position_size_usd = 3989.8734,
    realized_pnl = 0,
    sl_pct = 0.0057,
    sl_price = 28173.7,
    total_trades = 1,
    tp_pct = 0.0238,
    tp_price = 29009.3,
)
